# Handling multiple experiments
The INCAWrapper can create a model with data from multiple experiments. This is relevant when you wish to fit data from a parallel labelling experiment. The toy model example, which have worked with has two experiments. We will return the the simple toy model [1,2] that we also used in the Quick start guide (ref: ). In this notebook we will assume that you are familiar with the dataset, if not please refer to the quick start guide.

First, we will import the required python packages and the data.

In [1]:
import pandas as pd
import pathlib
import incawrapper
import ast

In [2]:
data_folder = pathlib.Path("./examples/Literature data/simple model")
tracers_data = pd.read_csv(data_folder / "tracers.csv", 
   converters={'atom_mdv':ast.literal_eval, 'atom_ids':ast.literal_eval} # a trick to read lists from csv
)
reactions_data = pd.read_csv(data_folder / "reactions.csv")
flux_data = pd.read_csv(data_folder / "flux_measurements.csv")
ms_data = pd.read_csv(data_folder / "ms_measurements.csv", 
   converters={'labelled_atom_ids': ast.literal_eval} # a trick to read lists from csv
)

We will take a closer look at the MS data.

In [3]:
ms_data

,experiment_id,met_id,ms_id,measurement_replicate,labelled_atom_ids,unlabelled_atoms,mass_isotope,intensity,intensity_std_error,time
0,exp1,F,F1,1,"[1, 2, 3]",NaN,0,0.0001,0.000002,0
1,exp1,F,F1,1,"[1, 2, 3]",NaN,1,0.8008,0.016016,0
2,exp1,F,F1,1,"[1, 2, 3]",NaN,2,0.1983,0.003966,0
3,exp1,F,F1,1,"[1, 2, 3]",NaN,3,0.0009,0.000018,0
4,exp2,F,F1,1,"[1, 2, 3]",NaN,0,0.0002,0.000002,0
5,exp2,F,F1,1,"[1, 2, 3]",NaN,1,0.7008,0.016016,0
6,exp2,F,F1,1,"[1, 2, 3]",NaN,2,0.1800,0.003966,0
7,exp2,F,F1,1,"[1, 2, 3]",NaN,3,0.0009,0.000018,0


We see that this data holds measurements of the ms_id (or ms fragment) from two different experiments, exp1 and exp2. If we have a closer look at the tracers data, we see that also this describes two labelling experiments.

In [4]:
tracers_data

,experiment_id,met_id,tracer_id,atom_ids,ratio,atom_mdv,enrichment
0,exp1,A,[2-13C]A,[2],1.0,"[0, 1]",1
1,exp2,A,"[1,2-13C]A","[1, 2]",0.5,"[0.05, 0.95]",1


To create an INCAScript with data from parallel labelling experiments simply specify the experiments in the `experiment_id` argument of the `create_inca_script_from_data` function.

In [5]:
incascript = incawrapper.create_inca_script_from_data(
    reactions_data=reactions_data,
    tracer_data=tracers_data,
    flux_measurements=flux_data, 
    ms_measurements=ms_data,
    experiment_ids=["exp1", "exp2"])

Now, we simply need to specify the options and which algorithms to run.

In [6]:
OUTPUT_FILE = data_folder / "simple_model_inca_multiple_exp.mat"
incascript.add_to_block('options', incawrapper.define_options(fit_starts=20, sim_na=False))
incascript.add_to_block('runner', incawrapper.define_runner(OUTPUT_FILE, run_estimate=True))

We can now run the INCAScript in INCA.

In [7]:
import dotenv
inca_directory = dotenv.get_key(dotenv.find_dotenv(), "INCA_base_directory")
incawrapper.run_inca(incascript, INCA_base_directory=inca_directory)

INCA script saved to /var/folders/z6/mxpxh4k56tv0h0ff41vmx7gdwtlpvp/T/tmptjx2o4yi/inca_script.m.
Starting MATLAB engine...
 
ms_exp1 = 1x1 msdata object
 
fields: atoms  id  [idvs]  more  on  state  
 
F1
 
 
ms_exp2 = 1x1 msdata object
 
fields: atoms  id  [idvs]  more  on  state  
 
F1
 
 
m = 1x1 model object
 
fields: [expts]  [mets]  notes  [options]  [rates]  [states]  
 
	5 reactions (6 fluxes)                                  
	6 states (3 balanced, 1 source, 2 sink and 0 unbalanced)
	6 metabolites                                           
	2 experiments                                           
 

                                         Directional 
 Iteration      Residual     Step-size    derivative        Lambda
     0       1.00011e+12
     1        9.9892e+11      0.000816     -7.59e+11       11092.6
     2       9.93976e+11       0.00261     -9.48e+11       11092.6
     3       1.63688e+08             1     -2.36e+09       11092.6
     4       8.05725e+07             

Finally, the results can be imported to python using the `INCAResults` as usual.

In [8]:
res = incawrapper.INCAResults(OUTPUT_FILE)

And one can inspect the results and analyze it as any other data.

In [9]:
res.fitdata.fitted_parameters

,type,id,eqn,val,std,lb,ub,unit,free,alf,chi2s,cont,cor,cov,vals,base
0,Net flux,R1,A -> B,1.000000e+01,0.000010,[],[],[],0,0.05,[],1,"[1.0, 0.00010674352435121925, 0.0, 5.286525244...","[9.999999960041972e-11, 6.687822951684241e-11,...",[],{'id': []}
1,Net flux,R2 net,B <-> D,6.687824e+00,0.062653,[],[],[],1,0.05,[],1,"[0.0001067435188136774, 1.0, 0.0, -0.999999987...","[6.687822604739546e-11, 0.003925422983263398, ...",[],{'id': []}
2,Exch flux,R2 exch,B <-> D,1.000000e-07,0.000000,[],[],[],1,0.05,[],1,"[0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]",[],{'id': []}
3,Net flux,R3,B -> C + E,1.656088e+00,0.031327,[],[],[],1,0.05,[],1,"[5.2865257985399465e-05, -0.9999999872625187, ...","[1.656088677651213e-11, -0.0019627114581925844...",[],{'id': []}
4,Net flux,R4,B + C -> D + E + E,1.656088e+00,0.031327,[],[],[],0,0.05,[],1,"[5.2865257985399465e-05, -0.9999999872625187, ...","[1.656088677651213e-11, -0.0019627114581925844...",[],{'id': []}
5,Net flux,R5,D -> F,8.343912e+00,0.031327,[],[],[],0,0.05,[],1,"[0.0002663522874548933, 0.9999999872625207, 0....","[8.343911282390759e-11, 0.001962711525070814, ...",[],{'id': []}
6,Norm,exp1 F1 exp1_F1_0_0_1 norm,[],9.990001e-01,0.016498,[],[],[],1,0.05,[],0,"[-1.9715049512208566e-12, 0.5164103260129809, ...","[-3.2526065174565133e-19, 0.000533791685561596...",[],{'id': []}
7,Norm,exp2 F1 exp2_F1_0_0_1 norm,[],9.523818e-02,0.000892,[],[],[],1,0.05,[],0,"[7.718240171440117e-13, 3.4992380087242814e-08...","[6.888177722179846e-21, 1.9566032458283252e-12...",[],{'id': []}


## References
[1] M. R. Antoniewicz, J. K. Kelleher, and G. Stephanopoulos, “Determination of confidence intervals of metabolic fluxes estimated from stable isotope measurements,” Metabolic Engineering, vol. 8, no. 4, pp. 324–337, Jul. 2006, doi: 10.1016/j.ymben.2006.01.004.

[2] M. R. Antoniewicz, J. K. Kelleher, and G. Stephanopoulos, “Elementary metabolite units (EMU): A novel framework for modeling isotopic distributions,” Metabolic Engineering, vol. 9, no. 1, pp. 68–86, Jan. 2007, doi: 10.1016/j.ymben.2006.09.001.
